# Init

In [1]:
import pandas as pd
import numpy as np
from helper.eda import (
    null_summary,
    visualize_data
)
from helper.feature_engineering import (
    create_datetime_features,
    create_lagged_features
)

In [2]:
# load datasets, set index and parse index as datetime

# 2023
df_2023_ancillary = pd.read_csv('data/raw/2023/Ancillary Volumes & Prices (4H).csv', parse_dates=True, sep=',', index_col=0, nrows=365*6)
df_2023_ancillary.index = pd.to_datetime(df_2023_ancillary.index, dayfirst=True)

df_2023_day_ahead_hourly = pd.read_csv('data/raw/2023/Day-Ahead Price (1H).csv', parse_dates=True, sep=',', index_col=0, nrows=365*24)
df_2023_day_ahead_hourly.index = pd.to_datetime(df_2023_day_ahead_hourly.index, dayfirst=True)

df_2023_day_ahead_half = pd.read_csv('data/raw/2023/Prices & Forecasts (HH).csv', parse_dates=True, sep=',', index_col=0, nrows=365*24*2)
df_2023_day_ahead_half.index = pd.to_datetime(df_2023_day_ahead_half.index, dayfirst=True)

# 2024
df_2024_ancillary = pd.read_csv('data/raw/2024/Ancillary Volumes & Prices (4H).csv', parse_dates=True, sep=',', index_col=0, nrows=365*6)
df_2024_ancillary.index = pd.to_datetime(df_2024_ancillary.index, dayfirst=True)

df_2024_day_ahead_hourly = pd.read_csv('data/raw/2024/Day-Ahead Price (1H).csv', parse_dates=True, sep=',', index_col=0, nrows=365*24)
df_2024_day_ahead_hourly.index = pd.to_datetime(df_2024_day_ahead_hourly.index, dayfirst=True)

df_2024_day_ahead_half = pd.read_csv('data/raw/2024/Prices & Forecasts (HH).csv', parse_dates=True, sep=',', index_col=0, nrows=365*24*2)
df_2024_day_ahead_half.index = pd.to_datetime(df_2024_day_ahead_half.index, dayfirst=True); df_2024_day_ahead_half

,National Demand Forecast (NDF) - GB (MW),"Day Ahead Price (EPEX half-hourly, local) - GB (£/MWh)"
GMT Time,,
2024-01-01 00:00:00,22500,36.10
2024-01-01 00:30:00,23100,52.00
2024-01-01 01:00:00,22663,69.80
2024-01-01 01:30:00,21851,35.10
2024-01-01 02:00:00,20993,31.20
...,...,...
2024-12-30 21:30:00,27703,92.17
2024-12-30 22:00:00,26136,87.93
2024-12-30 22:30:00,24652,55.00


In [3]:
# concatenate datasets 
print("Ancillary Dataframes")
df_ancillary = pd.concat([df_2023_ancillary, df_2024_ancillary])
print(
    'df_2023_ancillary:', df_2023_ancillary.shape,
    '\ndf_2024_ancillary:', df_2024_ancillary.shape,
    '\ndf_ancillary:', df_ancillary.shape
)

print("\nDay Ahead Hourly Dataframes")
df_day_ahead_hourly = pd.concat([df_2023_day_ahead_hourly, df_2024_day_ahead_hourly])
print(
    'df_2023_day_ahead_hourly:', df_2023_day_ahead_hourly.shape,
    '\ndf_2024_day_ahead_hourly:', df_2024_day_ahead_hourly.shape,
    '\ndf_day_ahead_hourly:', df_day_ahead_hourly.shape
)

print("\nDay Ahead Half Hourly Dataframes")
df_day_ahead_half = pd.concat([df_2023_day_ahead_half, df_2024_day_ahead_half])
print(
    'df_2023_day_ahead_half:', df_2023_day_ahead_half.shape,
    '\ndf_2024_day_ahead_half:', df_2024_day_ahead_half.shape,
    '\ndf_day_ahead_half:', df_day_ahead_half.shape
)

Ancillary Dataframes
df_2023_ancillary: (2190, 18) 
df_2024_ancillary: (2190, 18) 
df_ancillary: (4380, 18)

Day Ahead Hourly Dataframes
df_2023_day_ahead_hourly: (8760, 2) 
df_2024_day_ahead_hourly: (8760, 2) 
df_day_ahead_hourly: (17520, 2)

Day Ahead Half Hourly Dataframes
df_2023_day_ahead_half: (17520, 2) 
df_2024_day_ahead_half: (17520, 2) 
df_day_ahead_half: (35040, 2)


In [4]:
print(df_ancillary.info())
print(df_day_ahead_hourly.info())
print(df_day_ahead_half.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4380 entries, 2023-01-01 03:00:00 to 2024-12-30 23:00:00
Data columns (total 18 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Volume Requirements Forecast - DC-H - GB (MW)  4356 non-null   float64
 1   Volume Requirements Forecast - DC-L - GB (MW)  4356 non-null   float64
 2   Volume Requirements Forecast - DR-H - GB (MW)  4380 non-null   int64  
 3   Volume Requirements Forecast - DR-L - GB (MW)  4380 non-null   int64  
 4   Volume Requirements Forecast - DM-H - GB (MW)  4362 non-null   float64
 5   Volume Requirements Forecast - DM-L - GB (MW)  4380 non-null   int64  
 6   Ancillary Volume Accepted - DC-H - GB (MW)     4380 non-null   int64  
 7   Ancillary Volume Accepted - DC-L - GB (MW)     4380 non-null   int64  
 8   Ancillary Volume Accepted - DR-H - GB (MW)     4374 non-null   float64
 9   Ancillary Volume

# Preprocessing

## EDA

In [6]:
visualize_data(df_day_ahead_hourly, start_date='2023-01-01 01:00', end_date='2023-12-31 23:00', is_price=True)

In [5]:
visualize_data(df_ancillary, start_date='2023-10-01 01:00', end_date='2023-10-10 23:00', ln_y=False, columns=[
       'Volume Requirements Forecast - DC-H - GB (MW)',
       'Volume Requirements Forecast - DC-L - GB (MW)',
       # 'Volume Requirements Forecast - DR-H - GB (MW)',
       # 'Volume Requirements Forecast - DR-L - GB (MW)',
       # 'Volume Requirements Forecast - DM-H - GB (MW)',
       # 'Volume Requirements Forecast - DM-L - GB (MW)',
       'Ancillary Volume Accepted - DC-H - GB (MW)',
       'Ancillary Volume Accepted - DC-L - GB (MW)',
       # 'Ancillary Volume Accepted - DR-H - GB (MW)',
       # 'Ancillary Volume Accepted - DR-L - GB (MW)',
       # 'Ancillary Volume Accepted - DM-H - GB (MW)',
       # 'Ancillary Volume Accepted - DM-L - GB (MW)',
       'Ancillary Price - DC-H - GB (£/MW/h)',
       'Ancillary Price - DC-L - GB (£/MW/h)',
       # 'Ancillary Price - DR-H - GB (£/MW/h)',
       # 'Ancillary Price - DR-L - GB (£/MW/h)',
       # 'Ancillary Price - DM-H - GB (£/MW/h)',
       # 'Ancillary Price - DM-L - GB (£/MW/h)'
       ]
)

## Null values
Before EAC trend-break there were a lot of null values in DFR dataset (up to almost 10%). 

In [7]:
# EAC Introduction: 1st of November 2023
# A lot of null values before that date (try start_date='2023-01-01 00:00')
# After EAC there's virtually no nulls (try start_date='2023-11-01 00:00')
null_summary(df_ancillary, start_date='2023-11-01 00:00')

,Absolute Nulls,Relative Nulls (%)
Column,,
Volume Requirements Forecast - DC-H - GB (MW),18,0.7
Volume Requirements Forecast - DC-L - GB (MW),18,0.7
Volume Requirements Forecast - DR-H - GB (MW),0,0.0
Volume Requirements Forecast - DR-L - GB (MW),0,0.0
Volume Requirements Forecast - DM-H - GB (MW),0,0.0
Volume Requirements Forecast - DM-L - GB (MW),0,0.0
Ancillary Volume Accepted - DC-H - GB (MW),0,0.0
Ancillary Volume Accepted - DC-L - GB (MW),0,0.0
Ancillary Volume Accepted - DR-H - GB (MW),0,0.0


In [42]:
null_summary(df_day_ahead_hourly, start_date='2023-11-01 00:00')

,Absolute Nulls,Relative Nulls (%)
Column,,
"Day Ahead Price (N2EX, local) - GB (£/MWh)",1,0.01
"Day Ahead Price (EPEX, local) - GB (£/MWh)",0,0.00


In [44]:
null_summary(df_day_ahead_half, start_date='2023-11-01 00:00')

,Absolute Nulls,Relative Nulls (%)
Column,,
National Demand Forecast (NDF) - GB (MW),0,0.0
"Day Ahead Price (EPEX half-hourly, local) - GB (£/MWh)",0,0.0


## Model A

### Match 30-minute timeslots
I considered two options for aligning day-ahead market data with DFR 4-hour blocks: 
1. expanding DFR data to match half-hourly granularity. 
2. aggregating day-ahead data to match DFR blocks  

I chose to use both and model them seperately.

Model A: First option to preserve granular details, which are crucial for forecasting short-term prices in our small dataset. This approach helps maintain the integrity of highly granular signals essential for accurate forecasting. Aligning lowest granularity 30min day-ahead market data with DFR 4-Hour blocks by essentially copying the datapoints a number of times (x8 for DFR, x2 for hourly day-ahead). Also introduce a column to keep track of original datapoint.

Model B: This model uses aggregated data on a 4-hour block level to capture larger patterns and trends in the data. By aggregating, we can identify and leverage broader market movements and demand cycles that may not be apparent at a more granular level. This approach helps in understanding the overall dynamics of the market, which can be crucial for long-term forecasting and strategic decision-making. Aggregation can also smooth out short-term noise, providing a clearer picture of underlying trends that influence DFR prices.

In [5]:
df_ancillary_30 = pd.DataFrame(np.repeat(df_ancillary.values, repeats=8, axis=0), columns=df_ancillary.columns)
# Create the is_original column
is_original = np.tile([1] + [0]*7, len(df_ancillary))
df_ancillary_30['is_original'] = is_original
periods = 2*365*6*8 # 2 years of 4 hour daily blocks (6/day) with two 30min timeslots per hour
df_ancillary_30.index = pd.date_range(start='2023-01-01 03:00', periods=periods, freq='30min') # end='2024-12-31 02:30', 30mins before new year new timeslot
df_ancillary_30.value_counts('is_original')
df_ancillary_30

,Volume Requirements Forecast - DC-H - GB (MW),Volume Requirements Forecast - DC-L - GB (MW),Volume Requirements Forecast - DR-H - GB (MW),Volume Requirements Forecast - DR-L - GB (MW),Volume Requirements Forecast - DM-H - GB (MW),Volume Requirements Forecast - DM-L - GB (MW),Ancillary Volume Accepted - DC-H - GB (MW),Ancillary Volume Accepted - DC-L - GB (MW),Ancillary Volume Accepted - DR-H - GB (MW),Ancillary Volume Accepted - DR-L - GB (MW),Ancillary Volume Accepted - DM-H - GB (MW),Ancillary Volume Accepted - DM-L - GB (MW),Ancillary Price - DC-H - GB (£/MW/h),Ancillary Price - DC-L - GB (£/MW/h),Ancillary Price - DR-H - GB (£/MW/h),Ancillary Price - DR-L - GB (£/MW/h),Ancillary Price - DM-H - GB (£/MW/h),Ancillary Price - DM-L - GB (£/MW/h),is_original
2023-01-01 03:00:00,839.0,687.0,80.0,80.0,80.0,80.0,689.0,818.0,84.0,12.0,52.0,NaN,1.76,3.5,0.00,3.00,6.50,NaN,1
2023-01-01 03:30:00,839.0,687.0,80.0,80.0,80.0,80.0,689.0,818.0,84.0,12.0,52.0,NaN,1.76,3.5,0.00,3.00,6.50,NaN,0
2023-01-01 04:00:00,839.0,687.0,80.0,80.0,80.0,80.0,689.0,818.0,84.0,12.0,52.0,NaN,1.76,3.5,0.00,3.00,6.50,NaN,0
2023-01-01 04:30:00,839.0,687.0,80.0,80.0,80.0,80.0,689.0,818.0,84.0,12.0,52.0,NaN,1.76,3.5,0.00,3.00,6.50,NaN,0
2023-01-01 05:00:00,839.0,687.0,80.0,80.0,80.0,80.0,689.0,818.0,84.0,12.0,52.0,NaN,1.76,3.5,0.00,3.00,6.50,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 00:30:00,1245.0,1317.0,330.0,330.0,200.0,170.0,1319.0,1289.0,350.0,308.0,200.0,169.0,1.75,4.0,-2.05,10.25,0.45,5.22,0
2024-12-31 01:00:00,1245.0,1317.0,330.0,330.0,200.0,170.0,1319.0,1289.0,350.0,308.0,200.0,169.0,1.75,4.0,-2.05,10.25,0.45,5.22,0
2024-12-31 01:30:00,1245.0,1317.0,330.0,330.0,200.0,170.0,1319.0,1289.0,350.0,308.0,200.0,169.0,1.75,4.0,-2.05,10.25,0.45,5.22,0
2024-12-31 02:00:00,1245.0,1317.0,330.0,330.0,200.0,170.0,1319.0,1289.0,350.0,308.0,200.0,169.0,1.75,4.0,-2.05,10.25,0.45,5.22,0


In [6]:
df_day_ahead_hourly_30 = pd.DataFrame(np.repeat(df_day_ahead_hourly.values, repeats=2, axis=0), columns=df_day_ahead_hourly.columns)
# Create the is_original column
is_original = np.tile([1] + [0], len(df_day_ahead_hourly))
df_day_ahead_hourly_30['is_original'] = is_original
periods=2*365*24*2 # 2 years of 24 hour daily blocks with two 30min timeslots per hour
df_day_ahead_hourly_30.index = pd.date_range(start='2023-01-01 00:00', periods=periods, freq='30min') # end='2024-12-3O 23:30', 30mins before new year new timeslot
df_day_ahead_hourly_30.value_counts('is_original')
df_day_ahead_hourly_30

,"Day Ahead Price (N2EX, local) - GB (£/MWh)","Day Ahead Price (EPEX, local) - GB (£/MWh)",is_original
2023-01-01 00:00:00,13.01,60.0,1
2023-01-01 00:30:00,13.01,60.0,0
2023-01-01 01:00:00,25.05,49.5,1
2023-01-01 01:30:00,25.05,49.5,0
2023-01-01 02:00:00,0.99,18.0,1
...,...,...,...
2024-12-30 21:30:00,106.43,98.5,0
2024-12-30 22:00:00,90.40,84.5,1
2024-12-30 22:30:00,90.40,84.5,0
2024-12-30 23:00:00,72.00,62.7,1


In [7]:
df_day_ahead_half['is_original'] = 1
df_day_ahead_half.value_counts('is_original')

is_original
1    35040
Name: count, dtype: int64

In [8]:
df_ancillary_30.drop(columns=['is_original'], inplace=True)
df_day_ahead_hourly_30.drop(columns=['is_original'], inplace=True)
df_day_ahead_half.drop(columns=['is_original'], inplace=True)

### Subset for full-days in ancillary dataset (WRONG PLACE?)

In [9]:
# we now have all three datasets on a 30minute format, but we'll have to make the datetime indices match now
# the ancillary dataset is the smallest (least amount of rows) so we'll subset the other two datasets to match it
# we'll make use of the "day" (day starts at 23:00 on t-1 and ends at 23:00 on t) syntax of DFR markets
print("Amount of full days in set: ",df_ancillary_30.loc['2023-01-01 23:00':'2024-12-30 22:30'].shape[0]/8/6)
df_ancillary_resampled = df_ancillary_30.loc['2023-01-01 23:00':'2024-12-30 22:30']
# there's 729 full "day"s in the ancillary dataset, so we'll subset the other two datasets to match this
df_day_ahead_hourly_resampled = df_day_ahead_hourly_30.loc['2023-01-01 23:00':'2024-12-30 22:30']
df_day_ahead_half_resampled = df_day_ahead_half.loc['2023-01-01 23:00':'2024-12-30 22:30']

print(
    'df_ancillary_resampled:', df_ancillary_resampled.shape,
    '\ndf_day_ahead_hourly_resampled:', df_day_ahead_hourly_resampled.shape,
    '\ndf_day_ahead_half_resampled:', df_day_ahead_half_resampled.shape
)

Amount of full days in set:  729.0
df_ancillary_resampled: (34992, 18) 
df_day_ahead_hourly_resampled: (34992, 2) 
df_day_ahead_half_resampled: (34992, 2)


### Merge three datasets (WRONG PLACE?)

In [10]:
# we'll now merge the three datasets by the datetime index
df_merged_A = pd.concat([df_ancillary_resampled, df_day_ahead_hourly_resampled, df_day_ahead_half_resampled], axis=1)
df_merged_A.shape

(34992, 22)

## Model B

### Match 4-hour EFA blocks
To Do 

# Feature Engineering
Techniques included

| Technique              | Why                                                |
|------------------------|-----------------------------------------------------------------------|
| Holidays               | Captures the impact of holidays on energy consumption patterns.      |
| Datetime Features      | Incorporates temporal trends like hour of day, day of week, etc.     |
| Sinusoid Transform     | Models seasonal patterns and cyclical effects in the data.           |
| Summarization (Min, Max, Mean, Std) | Provides statistical insights into data distribution and variability. |
| Lagged Features        | Accounts for temporal dependencies and past influences on current data.|


In [11]:
df_merged_A.shape

(34992, 22)

## Model A

### Datetime features

In [14]:
date_df = create_datetime_features(df_merged_A)

,Volume Requirements Forecast - DC-H - GB (MW),Volume Requirements Forecast - DC-L - GB (MW),Volume Requirements Forecast - DR-H - GB (MW),Volume Requirements Forecast - DR-L - GB (MW),Volume Requirements Forecast - DM-H - GB (MW),Volume Requirements Forecast - DM-L - GB (MW),Ancillary Volume Accepted - DC-H - GB (MW),Ancillary Volume Accepted - DC-L - GB (MW),Ancillary Volume Accepted - DR-H - GB (MW),Ancillary Volume Accepted - DR-L - GB (MW),...,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,hour_of_day,halfhour_of_day,efa_block
2023-01-01 23:00:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,1,52,1,0,1,6,1,23,46,1
2023-01-01 23:30:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,1,52,1,0,1,6,1,23,47,1
2023-01-02 00:00:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,2,1,1,0,1,0,0,0,0,1
2023-01-02 00:30:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,2,1,1,0,1,0,0,0,1,1
2023-01-02 01:00:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,2,1,1,0,1,0,0,1,2,1


### Lagged features
- 2D half hourly (half hourly = all lagged)
- 1D hourly (hourly = 1 regular, 1 lagged)
- 7D ancillary prices / volumes 

In [16]:
# Create 2D lagged features for day ahead hourly prices, drop target because lagged
lag_half_df = create_lagged_features(df_merged_A, lag_days=[col for col in range(48,(7*48)+1, 48)], target='Day Ahead Price (EPEX half-hourly, local) - GB (£/MWh)', drop_target=True)

# Create 7D lagged features for ancillary prices, drop target because lagged
lag_DCH_df = create_lagged_features(df_merged_A, lag_days=[col for col in range(48,(7*48)+1, 48)], target='Ancillary Price - DC-H - GB (£/MW/h)', drop_target=True)
lag_DCL_df = create_lagged_features(df_merged_A, lag_days=[col for col in range(48,(7*48)+1, 48)], target='Ancillary Price - DC-L - GB (£/MW/h)', drop_target=True)
# DMH
# DML
# DRH
# DRL

# Create 7D lagged features for ancillary volumes, drop target because lagged
# DCH
# DCL
# DMH
# DML
# DRH
# DRL

# Create 1D lagged feature for day ahead hourly, don't drop target because regular

### Sinusoid

In [19]:
"To Do"

'To Do'

### Summarization

In [20]:
"To Do"

'To Do'

### Holidays

In [21]:
"To Do"

'To Do'

## Model B